# 5장. 케라스2로 구현하는 RNN(순환신경망)
## 5.2 문장을 판별하는 LSTM 구현 
LSTM을 이용하여 문장의 의미를 이해하는 예제를 구현

In [1]:
# set to use CPU
# import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

### 5.2.1 라이브러리 패키지 임포트
1. LSTM을 이용한 판별망 구현에 필요한 라이브러리 임포트

In [2]:
from __future__ import print_function

In [3]:
from keras.preprocessing import sequence
from keras.datasets import imdb
from keras import layers, models

### 5.2.2 데이터 준비     
2. 데이터는 케라스가 제공하는 공개 데이터인 IMDB를 사용합니다

In [5]:
class Data:
    def __init__(self, max_features=20000, maxlen=80):
        (self.x_train, self.y_train), (self.x_test, self.y_test) = \
            imdb.load_data(num_words=max_features)
        self.x_train = sequence.pad_sequences(self.x_train, maxlen=maxlen)
        self.x_test = sequence.pad_sequences(self.x_test, maxlen=maxlen)

### 5.2.3 모델링 
3. LSTM 모델링을 위한 클래스를 선언

In [6]:
class RNN_LSTM(models.Model):
    def __init__(self, max_features, maxlen):
        x = layers.Input((maxlen,))
        h = layers.Embedding(max_features, 128)(x)
        h = layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2)(h)
        y = layers.Dense(1, activation='sigmoid')(h)
        super().__init__(x, y)

        # try using different optimizers and different optimizer configs
        self.compile(loss='binary_crossentropy',
                    optimizer='adam', metrics=['accuracy'])

### 5.2.4 학습 및 성능 평가 
4. 학습 및 성능 평가를 담당할 머신 클래스

In [7]:
class Machine:
    def __init__(self,
                max_features=20000,
                maxlen=80):
        self.data = Data(max_features, maxlen)
        self.model = RNN_LSTM(max_features, maxlen)

    def run(self, epochs=3, batch_size=32):
        data = self.data
        model = self.model
        print('Training stage')
        print('==============')
        model.fit(data.x_train, data.y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 validation_data=(data.x_test, data.y_test))

        score, acc = model.evaluate(data.x_test, data.y_test,
                                   batch_size=batch_size)
        print('Test performance: accuracy={0}, loss={1}'.format(acc, score))

In [8]:
m = Machine()
m.run()

17465344/17464789 [==============================] - 2s 0us/step


<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/home/sjkim/anaconda3/envs/jax/lib/python3.9/site-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/home/sjkim/anaconda3/envs/jax/lib/python3.9/site-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or

NotImplementedError: Cannot convert a symbolic Tensor (lstm/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

---
### 전체 코드

In [19]:
# ① 라이브러리 임포트
from __future__ import print_function
from keras.preprocessing import sequence
from keras.datasets import imdb
from keras import layers, models

# ② 데이터 준비 
class Data:
    def __init__(self, max_features=20000, maxlen=80):
        (self.x_train, self.y_train), (self.x_test, self.y_test) = imdb.load_data(num_words=max_features)
        self.x_train = sequence.pad_sequences(self.x_train, maxlen=maxlen)
        self.x_test = sequence.pad_sequences(self.x_test, maxlen=maxlen)

# ③ 모델링
class RNN_LSTM(models.Model):
    def __init__(self, max_features, maxlen):
        x = layers.Input((maxlen,))
        h = layers.Embedding(max_features, 128)(x)
        h = layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2)(h)
        y = layers.Dense(1, activation='sigmoid')(h)
        super().__init__(x, y)

        # try using different optimizers and different optimizer configs
        self.compile(loss='binary_crossentropy',
                    optimizer='adam', metrics=['accuracy'])

# ④ 학습 및 성능 평가 
class Machine:
    def __init__(self,
                max_features=20000,
                maxlen=80):
        self.data = Data(max_features, maxlen)
        self.model = RNN_LSTM(max_features, maxlen)

    def run(self, epochs=3, batch_size=32):
        data = self.data
        model = self.model
        print('Training stage')
        print('==============')
        model.fit(data.x_train, data.y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 validation_data=(data.x_test, data.y_test))

        score, acc = model.evaluate(data.x_test, data.y_test,
                                   batch_size=batch_size)
        print('Test performance: accuracy={0}, loss={1}'.format(acc, score))

def main():
    m = Machine()
    m.run()

main()

Training stage
Epoch 1/3
782/782 [==============================] - 131s 167ms/step - loss: 0.4299 - accuracy: 0.7977 - val_loss: 0.3840 - val_accuracy: 0.8344
Epoch 2/3
782/782 [==============================] - 131s 168ms/step - loss: 0.2544 - accuracy: 0.8988 - val_loss: 0.3705 - val_accuracy: 0.8360
Epoch 3/3
782/782 [==============================] - 16s 21ms/step - loss: 0.4513 - accuracy: 0.8304
Test performance: accuracy=0.8304399847984314, loss=0.45125243067741394
